In [ ]:
# ==========================
# 1️⃣ 라이브러리 임포트
# ==========================
import os, sys, time, logging
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# ==========================
# 2️⃣ 로거 설정
# ==========================
sys.path.append(r"C:\ESG_Project1\util")
from logger import setup_logger
logger = setup_logger(__name__)

# ==========================
# 3️⃣ 데이터 로드 및 정규화
# ==========================
DATA_DIR = r"C:/ESG_Project1/file/merge_data"
train_df = pd.read_csv(os.path.join(DATA_DIR, "train_data.csv"), index_col=0, parse_dates=True)
test_df  = pd.read_csv(os.path.join(DATA_DIR, "test_data.csv"), index_col=0, parse_dates=True)
target_col = '합산발전량(MWh)'

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df[[target_col]].values)
test_scaled  = scaler.transform(test_df[[target_col]].values)

# ==========================
# 4️⃣ 시퀀스 생성 함수
# ==========================
def create_sequences(data, input_steps=168, output_steps=24):
    X, y = [], []
    for i in range(len(data) - input_steps - output_steps + 1):
        X.append(data[i:i+input_steps])
        y.append(data[i+input_steps:i+input_steps+output_steps])
    return np.array(X), np.array(y)

input_steps = 168
output_steps = 24
X_train, y_train = create_sequences(train_scaled, input_steps, output_steps)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

# ==========================
# 5️⃣ Seq2Seq CNN-LSTM 모델 정의
# ==========================
class Seq2SeqCNNLSTM(nn.Module):
    def __init__(self, input_size=1, conv_channels=[32,16], lstm_hidden=32, output_steps=24):
        super().__init__()
        self.conv1 = nn.Conv1d(input_size, conv_channels[0], kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(conv_channels[0])
        self.conv2 = nn.Conv1d(conv_channels[0], conv_channels[1], kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(conv_channels[1])
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.encoder_lstm = nn.LSTM(input_size=conv_channels[1], hidden_size=lstm_hidden, batch_first=True)
        self.decoder_lstm = nn.LSTM(input_size=1, hidden_size=lstm_hidden, batch_first=True)
        self.fc = nn.Linear(lstm_hidden, 1)
        self.output_steps = output_steps

    def forward(self, x, y=None, teacher_forcing_ratio=0.5):
        batch_size = x.size(0)
        x = x.transpose(1,2)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = x.transpose(1,2)

        _, (hidden, cell) = self.encoder_lstm(x)
        decoder_input = x[:,-1,0].unsqueeze(-1)
        outputs = []

        for t in range(self.output_steps):
            decoder_input = decoder_input.unsqueeze(1) if decoder_input.dim()==2 else decoder_input
            decoder_output, (hidden, cell) = self.decoder_lstm(decoder_input, (hidden, cell))
            out = self.fc(decoder_output).squeeze(1)
            outputs.append(out)
            if y is not None and np.random.rand() < teacher_forcing_ratio:
                decoder_input = y[:,t].unsqueeze(-1)
            else:
                decoder_input = out
        return torch.stack(outputs, dim=1)

# ==========================
# 6️⃣ 학습 준비
# ==========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Seq2SeqCNNLSTM(output_steps=output_steps).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scaler_amp = torch.cuda.amp.GradScaler()
dataset = TensorDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)

OUTPUT_DIR = r"C:/ESG_Project1/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)
checkpoint_path = os.path.join(OUTPUT_DIR, "seq2seq_cnn_lstm.pt")
loss_history_path = os.path.join(OUTPUT_DIR, "seq2seq_loss_history.npy")

# ==========================
# 7️⃣ 체크포인트 로드
# ==========================
start_epoch = 1
best_loss = np.inf
counter = 0
loss_history = []

if os.path.exists(checkpoint_path):
    logger.info("🔄 이전 체크포인트 발견, 모델 로드 후 재학습 시작")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler_amp.load_state_dict(checkpoint.get('scaler_amp', scaler_amp.state_dict()))
    start_epoch = checkpoint.get('epoch', 0) + 1
    best_loss = checkpoint.get('best_loss', best_loss)
    if os.path.exists(loss_history_path):
        loss_history = list(np.load(loss_history_path))
else:
    logger.info("🆕 새 모델 학습 시작")

# ==========================
# 8️⃣ 학습 루프
# ==========================
epochs = 150
early_patience = 15

for epoch in range(start_epoch, epochs+1):
    start_time = time.time()
    model.train()
    total_loss = 0
    teacher_ratio = max(0.3, 0.7 - 0.4*(epoch-1)/epochs)

    for step, (xb, yb) in enumerate(loader, 1):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            y_pred = model(xb, yb, teacher_forcing_ratio=teacher_ratio)
            loss = criterion(y_pred, yb)

        scaler_amp.scale(loss).backward()
        scaler_amp.step(optimizer)
        scaler_amp.update()
        total_loss += loss.item() * xb.size(0)

        if step % 50 == 0 or step == len(loader):
            logger.info(f"Epoch {epoch}/{epochs} Step {step}/{len(loader)} | Batch Loss: {loss.item():.6f} | TF Ratio: {teacher_ratio:.2f}")

    avg_loss = total_loss / len(loader.dataset)
    loss_history.append(avg_loss)
    elapsed = time.time() - start_time
    logger.info(f"Epoch {epoch}/{epochs} Complete | Avg Loss: {avg_loss:.6f} | Time: {elapsed:.1f}s | TF Ratio: {teacher_ratio:.2f}")

    # 체크포인트 저장
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_amp': scaler_amp.state_dict(),
        'best_loss': best_loss
    }, checkpoint_path)
    np.save(loss_history_path, np.array(loss_history))

    # EarlyStopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        counter = 0
    else:
        counter += 1
        if counter >= early_patience:
            logger.info(f"✅ Early stopping triggered at epoch {epoch} | No improvement in {early_patience} epochs")
            break

# ==========================
# 9️⃣ 테스트셋 예측 (Rolling Forecast)
# ==========================
model.eval()
rolling_input = train_scaled[-input_steps:].tolist()
predicted_scaled = []

with torch.no_grad():
    i = 0
    while i < len(test_scaled):
        steps_remaining = len(test_scaled) - i
        steps_to_predict = min(output_steps, steps_remaining)
        X_input = torch.tensor(rolling_input[-input_steps:], dtype=torch.float32).unsqueeze(0).to(device)
        y_pred = model(X_input, teacher_forcing_ratio=0.0).cpu().numpy().flatten()
        for step in range(steps_to_predict):
            rolling_input.append([y_pred[step]])
            predicted_scaled.append(y_pred[step])
        i += steps_to_predict

predicted_scaled = np.array(predicted_scaled).reshape(-1,1)
predicted_generation = scaler.inverse_transform(predicted_scaled)

# ==========================
# 🔟 성능 평가
# ==========================
y_true = test_df[[target_col]].values
rmse = np.sqrt(mean_squared_error(y_true, predicted_generation))
r2 = r2_score(y_true, predicted_generation)
logger.info(f"Test RMSE: {rmse:.2f} | R²: {r2:.4f}")

# ==========================
# ⓫ 결과 저장
# ==========================
result_df = pd.DataFrame({
    "날짜": test_df.index,
    "실제발전량(MWh)": y_true.flatten(),
    "예측발전량(MWh)": predicted_generation.flatten()
})
result_df["오차(MWh)"] = result_df["예측발전량(MWh)"] - result_df["실제발전량(MWh)"]
result_df.to_csv(os.path.join(OUTPUT_DIR, "predicted_generation.csv"), index=False)
logger.info(f"✅ 예측 결과 저장 완료: {OUTPUT_DIR}")


C:\Users\baesa\AppData\Local\Temp\ipykernel_5144\983595538.py:94: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_amp = torch.cuda.amp.GradScaler()
c:\ESG_Project1\311.venv\Lib\site-packages\torch\cuda\amp\grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(


[2025-10-23 13:41:48,425]✅ INFO - 🆕 새 모델 학습 시작


c:\ESG_Project1\311.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\baesa\AppData\Local\Temp\ipykernel_5144\983595538.py:140: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
c:\ESG_Project1\311.venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[2025-10-23 13:42:10,762]✅ INFO - Epoch 1/150 Step 50/2939 | Batch Loss: 0.001626 | TF Ratio: 0.70
[2025-10-23 13:42:18,752]✅ INFO - Epoch 1/150 Step 100/2939 | Batch Loss: 0.000308 | TF Ratio: 0.70
[2025-10-23 13:42:28,704]✅ INFO - Epoch 1/150 Step 150/2939 | Batch Loss: 0.002088 | TF Ratio: 0.70
[2025-10-23 13:42:35,961]✅ INFO - Epoch 1/150 Step 200/2939 | Batch Loss: 0.002401 | TF Ratio: 0.70
[2025-10-23 13:42:43,068]✅ INFO - Epoch 1/150 Step 250/2939 | Batch Loss: 0.001922 | TF Ratio: 0.70
[2025-10-23 13:42:49,263]✅ INFO - Epoch 1/150 Step 300/2939 | Batch Loss: 0.000937 | TF Ratio: 0.70
[2025-10-23 13:42:54,813]✅ INFO - Epoch 1/150 Step 350/2939 | Batch Loss: 0.000884 | TF Ratio: 0.70
[2025-10-23 13:43:00,901]✅ INFO - Epoch 1/150 Step 400/2939 | Batch Loss: 0.000728 | TF Ratio: 0.70
[2025-10-23 13:43:06,453]✅ INFO - Epoch 1/150 Step 450/2939 | Batch Loss: 0.000619 | TF Ratio: 0.70
[2025-10-23 13:43:11,737]✅ INFO - Epoch 1/150 Step 500/2939 | Batch Loss: 0.000417 | TF Ratio: 0.70
[